### **1. Paramètres**

In [1]:
# Importation des bibliothèques
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from functions_extract import (jointure_parcelle, jointure_pixel, 
                       extract_confidence_values, extract_lidar_values)

In [2]:
# Définition des paramètres
annees = [2017, 2018, 2019, 2020, 2021, 2022]  # Liste des années à traiter
zones = ['T30TYP', 'T30TYQ', 'T31TCJ', 'T31TGL', 'T31UEP']  # Liste des zones (tuiles)
lidar_metrics = ['grid_CC', 'grid_ENL', 'grid_MOCH', 'grid_PAI', 'grid_VCI'] # métriques lidar

# Chemins des répertoires et fichiers
output_dir_confidence = '../data_final/raster/confidence'
output_dir_lidar = '../data_final/raster/lidar' 
gpkg_parcelles_parcelle = '../data_final/vector/peupleraies_lidar_parcelle.gpkg'
gpkg_parcelles_pixel = '../data_final/vector/peupleraies_lidar_pixel.gpkg'
output_dir_csv = '../data_final/tableaux'  # Répertoire pour sauvegarder les fichiers CSV

# Création du répertoire de sortie pour les tableaux CSV s'il n'existe pas
os.makedirs(output_dir_csv, exist_ok=True)

# Chargement des vecteurs à partir des fichiers GeoPackage
peupleraies_parcelle = gpd.read_file(gpkg_parcelles_parcelle, layer='peupleraies_merged_parcelle')
peupleraies_pixel = gpd.read_file(gpkg_parcelles_pixel, layer='peupleraies_merged_pixel')

 ### **2. Extraction Raster de confidence** 

In [3]:
### **2. Extraction Raster de confidence** 
df_parcelle_all = []

# Boucle sur chaque zone pour extraire les valeurs
for zone in zones:
    conf_raster = os.path.join(output_dir_confidence, f"confidence_masked_{zone}.tif")
    df_conf = extract_confidence_values(conf_raster, annees, nodata=0)

    if df_conf is not None and not df_conf.empty:
        # Arrondi des coordonnées pour assurer la précision
        df_conf['x'] = df_conf['x'].round(2)
        df_conf['y'] = df_conf['y'].round(2)

        # Jointure spatiale à l'échelle parcelle
        df_merge = jointure_parcelle(df_conf, peupleraies_parcelle)

        if not df_merge.empty:
            df_parcelle_all.append(df_merge)

# Regroupement des résultats finaux pour les parcelles
if df_parcelle_all:
    df_parcelle_final = pd.concat(df_parcelle_all, ignore_index=True)

    # Calcul des statistiques globales
    print("\n*** Statistiques finales pour les parcelles ***")
    print(f"Nombre total de lignes : {len(df_parcelle_final)}")
    print(f"Nombre total de unique_ids : {df_parcelle_final['unique_id'].nunique()}")
    print("Nombre de lignes par département (source) :")
    print(df_parcelle_final.groupby('source').size())
    print("Nombre de unique_ids par département (source) :")
    print(df_parcelle_final.groupby('source')['unique_id'].nunique())

    # Exportation du CSV final
    df_parcelle_final.to_csv(os.path.join(output_dir_csv, 'df_parcelle.csv'), index=False)
    print("CSV final à l'échelle parcelle généré : df_parcelle.csv")



*** Statistiques finales pour les parcelles ***
Nombre total de lignes : 77617
Nombre total de unique_ids : 323
Nombre de lignes par département (source) :
source
dep10       15199
dep47       18061
dep73       22110
dep82_bb    10654
dep82_sp    11593
dtype: int64
Nombre de unique_ids par département (source) :
source
dep10        64
dep47        60
dep73        19
dep82_bb     42
dep82_sp    138
Name: unique_id, dtype: int64
CSV final à l'échelle parcelle généré : df_parcelle.csv


 ### **3. Extraction Raster de confidence + Lidar** 

In [4]:
### **2. Extraction Raster de confidence** 
df_parcelle_all = []

# Boucle sur chaque zone pour extraire les valeurs
for zone in zones:
    conf_raster = os.path.join(output_dir_confidence, f"confidence_masked_{zone}.tif")
    df_conf = extract_confidence_values(conf_raster, annees, nodata=0)

    if df_conf is not None and not df_conf.empty:
        # Arrondi des coordonnées pour assurer la précision
        df_conf['x'] = df_conf['x'].round(2)
        df_conf['y'] = df_conf['y'].round(2)

        # Jointure spatiale à l'échelle parcelle
        df_merge = jointure_parcelle(df_conf, peupleraies_parcelle)

        if not df_merge.empty:
            df_parcelle_all.append(df_merge)

# Regroupement des résultats finaux pour les parcelles
if df_parcelle_all:
    df_parcelle_final = pd.concat(df_parcelle_all, ignore_index=True)

    # Calcul des statistiques globales
    print("\n*** Statistiques finales pour les parcelles ***")
    print(f"Nombre total de lignes : {len(df_parcelle_final)}")
    print(f"Nombre total de unique_ids : {df_parcelle_final['unique_id'].nunique()}")
    print("Nombre de lignes par département (source) :")
    print(df_parcelle_final.groupby('source').size())
    print("Nombre de unique_ids par département (source) :")
    print(df_parcelle_final.groupby('source')['unique_id'].nunique())

    # Exportation du CSV final
    df_parcelle_final.to_csv(os.path.join(output_dir_csv, 'df_parcelle.csv'), index=False)
    print("CSV final à l'échelle parcelle généré : df_parcelle.csv")



*** Statistiques finales pour les parcelles ***
Nombre total de lignes : 77617
Nombre total de unique_ids : 323
Nombre de lignes par département (source) :
source
dep10       15199
dep47       18061
dep73       22110
dep82_bb    10654
dep82_sp    11593
dtype: int64
Nombre de unique_ids par département (source) :
source
dep10        64
dep47        60
dep73        19
dep82_bb     42
dep82_sp    138
Name: unique_id, dtype: int64
CSV final à l'échelle parcelle généré : df_parcelle.csv
